In [40]:
import os
import gc
import sys
import glob
import copy
import numpy as np
import pandas as pd
import netCDF4 as nc
import multiprocessing as mp
from datetime import datetime, timedelta
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import matplotlib.ticker as mticker

In [41]:
site_name = 'AU-How' #'AU-Tum' #

In [42]:
PLUMBER2_path  = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/nc_files/"
csv_file  = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/process1_output/TVeg_all_sites.csv"
nc_file1  = f"/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/nc_files/{site_name}.nc"
# nc_file2  = "/g/data/w97/mm3972/data/PLUMBER2/CABLE-POP-CN/AU-Tum_PLUMBER2_CN_out_cable.nc"
nc_file2  = f"/g/data/w97/mm3972/data/PLUMBER2/CABLE/{site_name}_out.nc"
# nc_file2  = "/g/data/w97/mm3972/data/PLUMBER2/GFDL/AU-Tum.nc"

## Check CABLE

In [37]:
df_csv   = pd.read_csv(f'{csv_file}',usecols=['site_name','model_CABLE'])
TVeg_csv = df_csv.loc[df_csv['site_name'] == site_name,'model_CABLE'].values/3600
TVeg_csv

array([ 7.08616944e-08,  8.80689722e-08,  2.53709444e-08, ...,
       -3.43349056e-08, -4.18009306e-08, -4.91823778e-08])

In [20]:
df_nc1    = nc.Dataset(nc_file1) 
# TVeg_nc1  = df_nc1.variables['CABLE-POP-CN_TVeg'][:]
# TVeg_nc1  = df_nc1.variables['GFDL_TVeg'][:]
TVeg_nc1  = df_nc1.variables['CABLE_TVeg'][:]
print(TVeg_nc1)

[ 7.0861688e-08  8.8068973e-08  2.5370944e-08 ... -3.4334903e-08
 -4.1800931e-08 -4.9182376e-08]


In [21]:
df_nc2    = nc.Dataset(nc_file2) 
TVeg_nc2  = df_nc2.variables['TVeg'][:,0,0]
print(TVeg_nc2)

[ 7.0861688e-08  8.8068973e-08  2.5370944e-08 ... -3.4334903e-08
 -4.1800931e-08 -4.9182376e-08]


## Check CABLE-POP-CN

In [10]:
nc_file2  = f"/g/data/w97/mm3972/data/PLUMBER2/CABLE-POP-CN/{site_name}_PLUMBER2_CN_out_cable.nc"

In [11]:
df_csv   = pd.read_csv(f'{csv_file}',usecols=['site_name','model_CABLE-POP-CN'])
TVeg_csv = df_csv.loc[df_csv['site_name'] == site_name,'model_CABLE-POP-CN'].values/3600
TVeg_csv

array([7.23939333e-10, 3.77015694e-10, 3.03094111e-10, ...,
       1.87614333e-10, 1.49413444e-10, 1.11387981e-10])

In [16]:
df_nc1    = nc.Dataset(nc_file1) 
# TVeg_nc1  = df_nc1.variables['CABLE-POP-CN_TVeg'][:]
# TVeg_nc1  = df_nc1.variables['GFDL_TVeg'][:]
TVeg_nc1  = df_nc1.variables['CABLE-POP-CN_TVeg'][:]
print(TVeg_nc1)

[7.2393935e-10 3.7701570e-10 3.0309411e-10 ... 1.8761434e-10 1.4941344e-10
 1.1138798e-10]


In [13]:
df_nc2    = nc.Dataset(nc_file2) 
TVeg_nc2  = df_nc2.variables['TVeg'][:,:,:]

In [14]:
TVeg_weight = TVeg_nc2[:,0,0]*0.21+TVeg_nc2[:,1,0]*0.14+TVeg_nc2[:,2,0]*0.58+TVeg_nc2[:,3,0]*0.07
  # 0.21,
  # 0.14,
  # 0.58,
  # 0.07 ;
print(TVeg_weight)

[1.7969032e-07 9.3568815e-08 7.5269931e-08 ... 4.6584574e-08 3.7118649e-08
 2.7686363e-08]


## Check process1 TVeg

In [43]:
file_in  = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/process1_output/TVeg_all_sites.csv"

In [44]:
df_csv   = pd.read_csv(f'{file_in}',usecols=['model_CABLE','model_CABLE-POP-CN','obs_SWdown','CABLE_EF'])

In [45]:
np.sum(np.where((df_csv['model_CABLE']<0) & (df_csv['obs_SWdown']>5) & (df_csv['CABLE_EF']>0),1,0))

17970

In [33]:
np.sum(np.where(df_csv['model_CABLE']<0,1,0))

2590938

In [8]:
np.sum(np.where(df_csv['model_CABLE']<-300,1,0))

2

In [21]:
np.sum(np.where(df_csv['model_CABLE']>800,1,0))

0

### Check Qle

In [46]:
file_in  = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/process1_output/TVeg_all_sites.csv"

In [47]:
df_csv1  = pd.read_csv(f'{file_in}',usecols=['model_CABLE','CABLE_EF','obs_SWdown'])

In [86]:
tmp = np.where(np.any([df_csv1['model_CABLE']>999.,df_csv1['model_CABLE']<-999.],axis=0), np.nan, df_csv1['model_CABLE'])
# np.sum(np.where((tmp<0) & (df_csv1['obs_SWdown']<5) & (df_csv1['CABLE_EF']>0),1,0))
min(tmp)

-0.53977686

In [77]:
np.sum(np.where((df_csv1['model_CABLE']<0) & (df_csv1['obs_SWdown']<5),1,0))

1448661

In [69]:
np.sum(np.where(df_csv1['model_CABLE']<0,1,0))

1893022

In [20]:
np.sum(np.where(np.isnan(df_csv1['model_CABLE']),1,0))

545

In [ ]:
np.sum(np.where(np.isnan(df_csv1['model_CABLE']),1,0))

## Check Process3 output

In [48]:
file_in  = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/process3_output/curves/raw_data_Qle_VPD_hourly_RM16_DT_EF_model_0-0.2.csv"
df_csv   = pd.read_csv(f'{file_in}',usecols=['model_CABLE','CABLE_EF','model_CABLE-POP-CN'])

In [26]:
# np.sum(np.where(df_csv['model_CABLE-POP-CN']<-300,1,0))

0

In [27]:
# np.sum(np.where(df_csv['model_CABLE-POP-CN']<0,1,0))

0

In [49]:
file_in  = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/process3_output/curves/raw_data_TVeg_VPD_hourly_RM16_DT_EF_model_0-0.2.csv"
df_csv1  = pd.read_csv(f'{file_in}',usecols=['model_CABLE','CABLE_EF','model_CABLE-POP-CN','obs_SWdown'])

In [50]:
np.sum(np.where((df_csv1['model_CABLE']<0) & (df_csv1['obs_SWdown']>5) & (df_csv1['CABLE_EF']>0),1,0))

0

In [11]:
np.sum(np.where(df_csv1['model_CABLE']<0,1,0))

8078

In [44]:
for i in np.arange(len(df_csv1)):
    if df_csv1['model_CABLE'][i] <0:
        print("df_csv['model_CABLE'][i]",df_csv['model_CABLE'][i],"df_csv1['model_CABLE'][i]",df_csv1['model_CABLE'][i],"df_csv1['CABLE_EF'][i]",df_csv1['CABLE_EF'][i],
              "df_csv1['obs_SWdown'][i]",df_csv1['obs_SWdown'][i],"df_csv1['obs_Tair'][i]",df_csv1['obs_Tair'][i],"df_csv1['VPD'][i]",df_csv1['VPD'][i])

df_csv['model_CABLE'][i] 0.040205956 df_csv1['model_CABLE'][i] -0.0835318149860555 df_csv1['CABLE_EF'][i] 0.0024922618 df_csv1['obs_SWdown'][i] 15.683 df_csv1['obs_Tair'][i] -2.0 df_csv1['VPD'][i] 0.0
df_csv['model_CABLE'][i] 0.8743639 df_csv1['model_CABLE'][i] -0.2932388206521544 df_csv1['CABLE_EF'][i] 0.0711538 df_csv1['obs_SWdown'][i] 32.3 df_csv1['obs_Tair'][i] 1.240000000000009 df_csv1['VPD'][i] 0.3531
df_csv['model_CABLE'][i] 1.6217353 df_csv1['model_CABLE'][i] -0.1042303978633399 df_csv1['CABLE_EF'][i] 0.112578414 df_csv1['obs_SWdown'][i] 10.38 df_csv1['obs_Tair'][i] 18.32000000000005 df_csv1['VPD'][i] 0.5854
df_csv['model_CABLE'][i] 0.53429055 df_csv1['model_CABLE'][i] -0.2832890919523765 df_csv1['CABLE_EF'][i] 0.030078 df_csv1['obs_SWdown'][i] 7.11 df_csv1['obs_Tair'][i] 14.571000000000026 df_csv1['VPD'][i] 0.1082
df_csv['model_CABLE'][i] 1.2127208 df_csv1['model_CABLE'][i] -0.095102229776073 df_csv1['CABLE_EF'][i] 0.09751617 df_csv1['obs_SWdown'][i] 39.78 df_csv1['obs_Tair'][